# Замер скорости и качества трекинговой системы

В этом ноутбуке замеряются мои лучшие по качеству детектор и трекер: детектор march-best (дообученная yolo11l) с трекером BoT-SORT с использованием RAFT (подробнее см. [`tracker.md`](tracker.md)).

In [1]:
experiment_name = "march-best_botsort-raft-s-128p-1upd-noups-med"

In [3]:
import pathlib
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torchvision
import torchvision.transforms.functional as F
import trackeval
import tqdm.notebook as tqdm
import ultralytics.trackers.utils

sns.set_theme(style="whitegrid")
DEVICE = "cuda"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
class RaftGMC:
    def __init__(self, model_size: str = "small", image_size: int = 128, num_flow_updates: int = 1) -> None:
        if model_size == "small":
            weights = torchvision.models.optical_flow.Raft_Small_Weights.DEFAULT
            self.model = torchvision.models.optical_flow.raft_small(weights=weights)
        elif model_size == "large":
            weights = torchvision.models.optical_flow.Raft_Large_Weights.DEFAULT
            self.model = torchvision.models.optical_flow.raft_large(weights=weights)
        else:
            raise ValueError

        self.model = self.model.to(DEVICE).eval()
        self.transforms = weights.transforms()
        self.image_size = image_size
        self.num_flow_updates = num_flow_updates
        self.last_frame = None

    def apply(self, raw_frame: np.ndarray, detections: list = None) -> np.ndarray:
        """
        `raw_frame`: frame with shape [H, W, C].
        `detections`: unused for RAFT.
        Returns a 2x3 homography matrix.
        """
        raw_frame = torch.tensor(raw_frame).permute(2, 0, 1)[None, :]    # [1, C, H, W]
        raw_frame = raw_frame[:, :, :, 10:-10]

        scale_factor = raw_frame.shape[2] / self.image_size
        raw_frame = F.resize(raw_frame, size=self.image_size, antialias=False)    # [1, C, h, w]
        if self.last_frame is None:
            self.last_frame = raw_frame
            return np.eye(2, 3)

        frame1, frame2 = self.transforms(self.last_frame, raw_frame)
        flows = self.model(    # list[num_flow_updates] of [1, 2, h, w] (or h/8, w/8 with no upsampling)
            frame1.to(DEVICE), frame2.to(DEVICE), num_flow_updates=self.num_flow_updates
        )
        medians = flows[-1].reshape(2, -1).median(dim=1).values.cpu().numpy()    # [2]
        hom = np.eye(2, 3)
        hom[:, 2] = medians * scale_factor

        self.last_frame = raw_frame
        return hom

    def reset_params(self) -> None:
        self.last_frame = None


def get_gmc(method: str):
    if method == "raft":
        return RaftGMC()
    else:
        return ultralytics.trackers.utils.gmc.GMC(method=method)


def scale_flow(flow, **kwargs):
    return flow * 8


ultralytics.trackers.bot_sort.GMC = get_gmc
torchvision.models.optical_flow.raft.upsample_flow = scale_flow

In [5]:
botsort_cfg = """
# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLO tracker settings for BoT-SORT tracker https://github.com/NirAharon/BoT-SORT

# Thresholds are set to their values BEFORE the Oct 2024 commit decreasing them:
# https://github.com/ultralytics/ultralytics/commit/aabd0136ec40223cf423847635a2a4de95bba63d
# They seem to work better for this task.

tracker_type: botsort # tracker type, ['botsort', 'bytetrack']
track_high_thresh: 0.5 # threshold for the first association
track_low_thresh: 0.1 # threshold for the second association
new_track_thresh: 0.6 # threshold for init new track if the detection does not match any tracks
track_buffer: 30 # buffer to calculate the time when to remove tracks
match_thresh: 0.8 # threshold for matching tracks
fuse_score: True # Whether to fuse confidence scores with the iou distances before matching
# min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)

# BoT-SORT settings
gmc_method: raft # method of global motion compensation
# ReID model related thresh
proximity_thresh: 0.5 # minimum IoU for valid match with ReID
appearance_thresh: 0.25 # minimum appearance similarity for ReID
with_reid: False
model: auto # uses native features if detector is YOLO else yolo11n-cls.pt
"""
with open("custom-botsort.yaml", "w") as fout:
    fout.write(botsort_cfg)

In [6]:
TIME_KEYS = ["preprocess", "postprocess", "inference"]

data_path = "../input/sportsmot-small/images/val"

model_path = "../input/player-tracking-models/detector/spm-yolo-l-lr0-0.0005.pt"

botsort_cfg_path = "custom-botsort.yaml"

images_dirs = sorted(pathlib.Path(data_path).iterdir())
n_frames_total = sum(len(list((images_dir / "img1").iterdir())) for images_dir in images_dirs)
pbar = tqdm.tqdm(total=n_frames_total)

outputs = {}
times_ms = {key: [] for key in TIME_KEYS + ["wall"]}

for images_dir in images_dirs:
    model = ultralytics.YOLO(model_path)
    outputs[images_dir.stem] = []

    for frame_id, image_path in enumerate(sorted((images_dir / "img1").iterdir())):
        image = cv2.imread(image_path)
        start = time.perf_counter()
        result = model.track(image, tracker=botsort_cfg_path, verbose=False, persist=True)
        end = time.perf_counter()
        if frame_id > 0:    # 1st frame can be much slower due to setup
            times_ms["wall"].append((end - start) * 1000)
            for key in TIME_KEYS:
                times_ms[key].append(result[0].speed[key])

        boxes = result[0].boxes
        if boxes.id is not None and boxes.xyxy is not None and boxes.conf is not None:
            for player_id, (x1, y1, x2, y2), conf in zip(boxes.id, boxes.xyxy, boxes.conf):
                outputs[images_dir.stem].append((
                    frame_id + 1,  # 1-indexed frame number
                    round(player_id.item()),  # player id (1-indexed already)
                    x1.item(),  # left
                    y1.item(),  # top
                    (x2 - x1).item(),  # width
                    (y2 - y1).item(),  # height
                    conf.item(),  # confidence
                    -1, -1, -1  # unused
                ))
        pbar.update()

pbar.close()

  0%|          | 0/26970 [00:00<?, ?it/s]

Downloading: "https://download.pytorch.org/models/raft_small_C_T_V2-01064c6d.pth" to /root/.cache/torch/hub/checkpoints/raft_small_C_T_V2-01064c6d.pth

100%|██████████| 3.82M/3.82M [00:00<00:00, 90.4MB/s]


In [7]:
for key, times in times_ms.items():
    mean = np.mean(times)
    std = np.std(times)
    print(f"{key}: {mean:.1f} ± {std:.1f} ms / frame")

preprocess: 1.4 ± 0.1 ms / frame
postprocess: 1.3 ± 0.1 ms / frame
inference: 16.5 ± 0.8 ms / frame
wall: 32.3 ± 2.8 ms / frame


In [8]:
out_dir = pathlib.Path("predictions/sportsmot-val") / experiment_name / "data"
out_dir.mkdir(parents=True, exist_ok=True)

for clip_name, lines in outputs.items():
    text = "".join(", ".join(map(str, line)) + "\n" for line in lines)
    (out_dir / clip_name).with_suffix(".txt").write_text(text)

In [9]:
# trackeval hack, these are old aliases that are now deprecated
np.float = float
np.int = int
np.bool = bool

dataset_config = {
    "GT_FOLDER": "../input/sportsmot-trackeval-ground-truths",
    "TRACKERS_FOLDER": "predictions",
    "OUTPUT_FOLDER": "trackeval-results",
    "BENCHMARK": "sportsmot",
    "SPLIT_TO_EVAL": "val"
}

evaluator = trackeval.Evaluator()
datasets = [trackeval.datasets.MotChallenge2DBox(dataset_config)]
metrics = [
    trackeval.metrics.HOTA(),
    trackeval.metrics.CLEAR(),
    trackeval.metrics.Identity(),
    trackeval.metrics.VACE()
]

evaluator.evaluate(datasets, metrics);


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /usr/local/lib/python3.11/dist-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
GT_FOLDER            : ../input/sportsmot-trackeval-ground-truths
TRACKERS_FOLDER      : predictions                   
OUTPUT_FOLDER        : trackeval-results             
BENCH

<Figure size 640x480 with 0 Axes>